In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


In [13]:
df = pd.read_csv('/workspaces/codespaces-jupyter/Iris.csv')

In [14]:
df

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [16]:
df['class'] = df['class'].map({'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2})

In [18]:
df.head()

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [30]:
## import and do train test split
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
knn1 = KNeighborsClassifier(n_neighbors=2)
knn2 = KNeighborsClassifier(n_neighbors=3)
knn3 = KNeighborsClassifier(n_neighbors=4)
knn4 = KNeighborsClassifier(n_neighbors=5)

voting_clf = VotingClassifier(estimators=[('knn1', knn1), ('knn2', knn2), ('knn3', knn3), ('knn4', knn4)], voting='hard')


In [42]:
import warnings
warnings.filterwarnings("ignore")

## now train and log test results for voting classifier of knn's with different train test split sizes
df_results = pd.DataFrame(columns=['model', 'split_size', 'accuracy', 'precision', 'recall', 'f1'])
for s in [0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
    X_train, X_test, y_train, y_test = train_test_split(df.drop('class', axis=1), df['class'], train_size=s, random_state=42)
    voting_clf.fit(X_train, y_train)
    y_pred = voting_clf.predict(X_test)
    print(accuracy_score(y_test, y_pred))
    #log the final results in an dataframe
    df_results = df_results.append({'model':'voting_clf', 'split_size':s, 'accuracy':accuracy_score(y_test, y_pred), 'precision':precision_score(y_test, y_pred, average='weighted'), 'recall':recall_score(y_test, y_pred, average='weighted'), 'f1':f1_score(y_test, y_pred, average='weighted')}, ignore_index=True)

#save results to csv
df_results.to_csv('knn_voting_results.csv', index=False)
df_results


0.9523809523809523
0.9333333333333333
0.9466666666666667
0.9833333333333333
1.0
1.0


,model,split_size,accuracy,precision,recall,f1
0,voting_clf,0.3,0.952381,0.952607,0.952381,0.952358
1,voting_clf,0.4,0.933333,0.938047,0.933333,0.932706
2,voting_clf,0.5,0.946667,0.954568,0.946667,0.946260
3,voting_clf,0.6,0.983333,0.984167,0.983333,0.983309
4,voting_clf,0.7,1.000000,1.000000,1.000000,1.000000
5,voting_clf,0.8,1.000000,1.000000,1.000000,1.000000


In [43]:
##Do the same as above with adaboost
from sklearn.ensemble import BaggingClassifier

df_results = pd.DataFrame(columns=['split_size', 'model', 'accuracy', 'precision', 'recall', 'f1'])
for s in [0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
    for i in [2, 3, 4, 5]:
        X_train, X_test, y_train, y_test = train_test_split(df.drop('class', axis=1), df['class'], train_size=s, random_state=42)
        bagging_clf = BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=i), n_estimators=10, max_samples=0.5, max_features=0.5, bootstrap=True, bootstrap_features=False, n_jobs=-1, random_state=42)
        bagging_clf.fit(X_train, y_train)
        y_pred = bagging_clf.predict(X_test)
        print(accuracy_score(y_test, y_pred))
        #log the final results in an dataframe
        df_results = df_results.append({'model':'boosting_knn_'+str(i), 'split_size':s, 'accuracy':accuracy_score(y_test, y_pred), 'precision':precision_score(y_test, y_pred, average='weighted'), 'recall':recall_score(y_test, y_pred, average='weighted'), 'f1':f1_score(y_test, y_pred, average='weighted')}, ignore_index=True)

0.9428571428571428
0.9333333333333333
0.9142857142857143
0.9047619047619048
0.9444444444444444
0.9333333333333333
0.9333333333333333
0.9444444444444444
0.92
0.9333333333333333
0.92
0.92
0.95
0.9666666666666667
0.9666666666666667
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [44]:
df_results.to_csv('boosting_knn.csv')
df_results

,split_size,model,accuracy,precision,recall,f1
0,0.3,boosting_knn_2,0.942857,0.951648,0.942857,0.942282
1,0.3,boosting_knn_3,0.933333,0.945000,0.933333,0.932436
2,0.3,boosting_knn_4,0.914286,0.932653,0.914286,0.912416
3,0.3,boosting_knn_5,0.904762,0.926910,0.904762,0.902209
4,0.4,boosting_knn_2,0.944444,0.947162,0.944444,0.944110
5,0.4,boosting_knn_3,0.933333,0.938047,0.933333,0.932706
6,0.4,boosting_knn_4,0.933333,0.938047,0.933333,0.932706
7,0.4,boosting_knn_5,0.944444,0.947162,0.944444,0.944110
8,0.5,boosting_knn_2,0.920000,0.936552,0.920000,0.918615
9,0.5,boosting_knn_3,0.933333,0.945238,0.933333,0.932536


In [45]:
## do majority voting ensemble using svm with different kernels
from sklearn.svm import SVC

svm1 = SVC(kernel='linear')
svm2 = SVC(kernel='poly')
svm3 = SVC(kernel='rbf')
svm4 = SVC(kernel='sigmoid')
voting_clf = VotingClassifier(estimators=[('svm1', svm1), ('svm2', svm2), ('svm3', svm3), ('svm4', svm4)], voting='hard')

df_results = pd.DataFrame(columns=['split_size', 'model', 'accuracy', 'precision', 'recall', 'f1'])
for s in [0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
    X_train, X_test, y_train, y_test = train_test_split(df.drop('class', axis=1), df['class'], train_size=s, random_state=42)
    voting_clf.fit(X_train, y_train)
    y_pred = voting_clf.predict(X_test)
    print(accuracy_score(y_test, y_pred))
    #log the final results in an dataframe
    df_results = df_results.append({'model':'svm_voting', 'split_size':s, 'accuracy':accuracy_score(y_test, y_pred), 'precision':precision_score(y_test, y_pred, average='weighted'), 'recall':recall_score(y_test, y_pred, average='weighted'), 'f1':f1_score(y_test, y_pred, average='weighted')}, ignore_index=True)


0.9714285714285714
0.9888888888888889
0.96
1.0
1.0
1.0


In [46]:
#save results to csv
df_results.to_csv('svm_voting_results.csv', index=False)
df_results

,split_size,model,accuracy,precision,recall,f1
0,0.3,svm_voting,0.971429,0.973878,0.971429,0.971388
1,0.4,svm_voting,0.988889,0.989300,0.988889,0.988896
2,0.5,svm_voting,0.960000,0.964615,0.960000,0.959829
3,0.6,svm_voting,1.000000,1.000000,1.000000,1.000000
4,0.7,svm_voting,1.000000,1.000000,1.000000,1.000000
5,0.8,svm_voting,1.000000,1.000000,1.000000,1.000000


In [47]:
## do the same as above with adaboost
##Do the same as above with adaboost
from sklearn.ensemble import BaggingClassifier

df_results = pd.DataFrame(columns=['split_size', 'model', 'accuracy', 'precision', 'recall', 'f1'])
for s in [0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
    for i in ['linear', 'poly', 'rbf', 'sigmoid']:
        X_train, X_test, y_train, y_test = train_test_split(df.drop('class', axis=1), df['class'], train_size=s, random_state=42)
        bagging_clf = BaggingClassifier(base_estimator=SVC(kernel = i), n_estimators=10, max_samples=0.5, max_features=0.5, bootstrap=True, bootstrap_features=False, n_jobs=-1, random_state=42)
        bagging_clf.fit(X_train, y_train)
        y_pred = bagging_clf.predict(X_test)
        print(accuracy_score(y_test, y_pred))
        #log the final results in an dataframe
        df_results = df_results.append({'model':'boosting_svm_'+i, 'split_size':s, 'accuracy':accuracy_score(y_test, y_pred), 'precision':precision_score(y_test, y_pred, average='weighted'), 'recall':recall_score(y_test, y_pred, average='weighted'), 'f1':f1_score(y_test, y_pred, average='weighted')}, ignore_index=True)
##save results to csv
df_results.to_csv('boosting_svm.csv')
df_results

0.9333333333333333
0.9047619047619048
0.8476190476190476
0.3142857142857143
0.9444444444444444
0.9111111111111111
0.9333333333333333
0.08888888888888889
0.9733333333333334
0.9466666666666667
0.9733333333333334
0.30666666666666664
1.0
1.0
0.9666666666666667
0.2
1.0
1.0
1.0
0.13333333333333333
1.0
0.9666666666666667
0.9666666666666667
0.3


,split_size,model,accuracy,precision,recall,f1
0,0.3,boosting_svm_linear,0.933333,0.939088,0.933333,0.932856
1,0.3,boosting_svm_poly,0.904762,0.918312,0.904762,0.903107
2,0.3,boosting_svm_rbf,0.847619,0.882765,0.847619,0.839619
3,0.3,boosting_svm_sigmoid,0.314286,0.098776,0.314286,0.150311
4,0.4,boosting_svm_linear,0.944444,0.947162,0.944444,0.944110
5,0.4,boosting_svm_poly,0.911111,0.915152,0.911111,0.910275
6,0.4,boosting_svm_rbf,0.933333,0.938047,0.933333,0.932706
7,0.4,boosting_svm_sigmoid,0.088889,0.071518,0.088889,0.076227
8,0.5,boosting_svm_linear,0.973333,0.975467,0.973333,0.973283
9,0.5,boosting_svm_poly,0.946667,0.954568,0.946667,0.946260
